In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%matplotlib inline

In [2]:
import re
import nltk 
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import spacy

In [3]:
import textacy
import csv

In [5]:
df = pd.read_csv('fix1.txt', delimiter='\t', encoding="utf-8")

In [6]:
df

,date,title,content,close_price,open_price,polarity,neg,pos
0,2022-09-16T14:00:00+00:00,Why Warren Buffett Loves Apple,"With a market capitalization of $2.5 trillion,...",149.770004,151.210007,0.599,0.000,0.129
1,2022-09-16T13:49:38+00:00,Big Tech stocks face difficulty beyond rising ...,Yahoo Finance Live anchors discuss how inflati...,149.740005,151.210007,0.052,0.128,0.138
2,2022-09-16T13:35:41+00:00,Dow Jones Sells Off As FedEx Plunges 23% On Pr...,The Dow Jones Industrial Average dived 350 poi...,149.740005,151.210007,0.402,0.102,0.217
3,2022-09-16T13:34:07+00:00,ARM listing: Why UK is desperate for Cambridge...,"Arm, which focuses on semiconductor and softwa...",149.750000,151.210007,0.250,0.040,0.048
4,2022-09-16T12:53:41+00:00,Dow Jones Futures Sell Off As FedEx Plunges 20...,Dow Jones futures dropped 350 points Friday mo...,149.750000,151.210007,0.402,0.107,0.227
...,...,...,...,...,...,...,...,...
1826,2022-08-05T21:20:00+00:00,Advertising Is Still Going Strong. Apple Wants...,A recent job listing suggests Apple is interes...,165.350006,163.210007,0.557,0.000,0.204
1827,2022-08-05T21:18:07+00:00,Dow Jones Futures: Market Rally Resilient; App...,The market rally showed resilience after Frida...,165.350006,163.210007,0.000,0.000,0.000
1828,2022-08-05T21:02:17+00:00,President Xi may use Speaker Pelosi's visit to...,"Susan Shirk, UC San Diego Research Professor a...",165.350006,163.210007,0.992,0.038,0.096
1829,2022-08-05T20:45:00+00:00,The Best Multiport Wall Adapters of 2022,If you're tired of having all your gadgets spr...,165.350006,163.210007,-0.296,0.165,0.094


In [24]:
L = []
for x in range(0, df.shape[0]):
    if df["close_price"].iloc[x] > df["open_price"].iloc[x]:
        L.append(int(1))
    else:
        L.append(int(0))
        
df["movement"] = L

In [27]:
df = df.drop(["close_price", "open_price"], axis = 1)

In [28]:
df

,date,title,content,polarity,neg,pos,movement
0,2022-09-16T14:00:00+00:00,Why Warren Buffett Loves Apple,"With a market capitalization of $2.5 trillion,...",0.599,0.000,0.129,0
1,2022-09-16T13:49:38+00:00,Big Tech stocks face difficulty beyond rising ...,Yahoo Finance Live anchors discuss how inflati...,0.052,0.128,0.138,0
2,2022-09-16T13:35:41+00:00,Dow Jones Sells Off As FedEx Plunges 23% On Pr...,The Dow Jones Industrial Average dived 350 poi...,0.402,0.102,0.217,0
3,2022-09-16T13:34:07+00:00,ARM listing: Why UK is desperate for Cambridge...,"Arm, which focuses on semiconductor and softwa...",0.250,0.040,0.048,0
4,2022-09-16T12:53:41+00:00,Dow Jones Futures Sell Off As FedEx Plunges 20...,Dow Jones futures dropped 350 points Friday mo...,0.402,0.107,0.227,0
...,...,...,...,...,...,...,...
1826,2022-08-05T21:20:00+00:00,Advertising Is Still Going Strong. Apple Wants...,A recent job listing suggests Apple is interes...,0.557,0.000,0.204,1
1827,2022-08-05T21:18:07+00:00,Dow Jones Futures: Market Rally Resilient; App...,The market rally showed resilience after Frida...,0.000,0.000,0.000,1
1828,2022-08-05T21:02:17+00:00,President Xi may use Speaker Pelosi's visit to...,"Susan Shirk, UC San Diego Research Professor a...",0.992,0.038,0.096,1
1829,2022-08-05T20:45:00+00:00,The Best Multiport Wall Adapters of 2022,If you're tired of having all your gadgets spr...,-0.296,0.165,0.094,1


In [7]:
nlp = spacy.load("en_core_web_sm")

In [12]:
def preprocessing(mess):
    nopunc = [char.lower() for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    details = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    #details = [word for word in details if not word in set(surplus_words)]
    details = ' '.join(details)
    fin = []
    fin.append(details)
    fin = list(set(fin))
    return fin
    

In [13]:
df['content'].apply(preprocessing)

0       [market capitalization 25 trillion apple nasda...
1       [yahoo finance live anchors discuss inflation ...
2       [dow jones industrial average dived 350 points...
3       [arm focuses semiconductor software design own...
4       [dow jones futures dropped 350 points friday m...
                              ...                        
1826    [recent job listing suggests apple interested ...
1827    [market rally showed resilience fridays jobs r...
1828    [susan shirk uc san diego research professor 2...
1829    [youre tired gadgets spread across home taking...
1830    [bloomberg amazoncom inc barrelling ahead aggr...
Name: content, Length: 1831, dtype: object

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
import tensorflow as tf

In [16]:
ann = tf.keras.models.Sequential()

In [17]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [18]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [19]:
ann.add(tf.keras.layers.Dense(units=6, activation='softmax'))

In [20]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [21]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [30]:
cv = CountVectorizer()
Z = []
for i in range(0, df.shape[0]):
    s = ''.join(preprocessing(df['content'][i]))
    Z.append(s)
X = cv.fit_transform(Z)
y = df['movement']

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.30,random_state=52)

In [32]:
X_train = X_train.toarray()
type(X_test)

scipy.sparse.csr.csr_matrix

In [33]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', mode='max', verbose=1, patience=200)

In [34]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 500, verbose = 1)

Epoch 1/500
41/41 [==============================] - 3s 12ms/step - loss: 0.6837 - accuracy: 0.5597
Epoch 2/500
41/41 [==============================] - 0s 11ms/step - loss: 0.6618 - accuracy: 0.5597
Epoch 3/500
41/41 [==============================] - 0s 12ms/step - loss: 0.6185 - accuracy: 0.6347
Epoch 4/500
41/41 [==============================] - 0s 11ms/step - loss: 0.5711 - accuracy: 0.7572
Epoch 5/500
41/41 [==============================] - 0s 12ms/step - loss: 0.5287 - accuracy: 0.7931
Epoch 6/500
41/41 [==============================] - 0s 12ms/step - loss: 0.4744 - accuracy: 0.9110
Epoch 7/500
41/41 [==============================] - 0s 11ms/step - loss: 0.4340 - accuracy: 0.9641
Epoch 8/500
41/41 [==============================] - 0s 12ms/step - loss: 0.3976 - accuracy: 0.9797
Epoch 9/500
41/41 [==============================] - 0s 11ms/step - loss: 0.3667 - accuracy: 0.9852
Epoch 10/500
41/41 [==============================] - 0s 11ms/step - loss: 0.3399 - accuracy: 0.9891

41/41 [==============================] - 0s 11ms/step - loss: 0.0476 - accuracy: 0.9899
Epoch 163/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0480 - accuracy: 0.9899
Epoch 164/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0474 - accuracy: 0.9899
Epoch 165/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0469 - accuracy: 0.9899
Epoch 166/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0469 - accuracy: 0.9899
Epoch 167/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0469 - accuracy: 0.9899
Epoch 168/500
41/41 [==============================] - 0s 9ms/step - loss: 0.0466 - accuracy: 0.9899
Epoch 169/500
41/41 [==============================] - 0s 10ms/step - loss: 0.0467 - accuracy: 0.9899
Epoch 170/500
41/41 [==============================] - 0s 10ms/step - loss: 0.0469 - accuracy: 0.9899
Epoch 171/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0467 - accuracy: 0.

41/41 [==============================] - 0s 12ms/step - loss: 0.0549 - accuracy: 0.9867
Epoch 323/500
41/41 [==============================] - 0s 12ms/step - loss: 0.0546 - accuracy: 0.9867
Epoch 324/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0547 - accuracy: 0.9867
Epoch 325/500
41/41 [==============================] - 0s 10ms/step - loss: 0.0548 - accuracy: 0.9867
Epoch 326/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0545 - accuracy: 0.9867
Epoch 327/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0547 - accuracy: 0.9867
Epoch 328/500
41/41 [==============================] - 0s 10ms/step - loss: 0.0545 - accuracy: 0.9867
Epoch 329/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0545 - accuracy: 0.9867
Epoch 330/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0543 - accuracy: 0.9867
Epoch 331/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0543 - accuracy: 0

41/41 [==============================] - 0s 11ms/step - loss: 0.0521 - accuracy: 0.9867
Epoch 483/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0522 - accuracy: 0.9867
Epoch 484/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0521 - accuracy: 0.9867
Epoch 485/500
41/41 [==============================] - 0s 11ms/step - loss: 0.0521 - accuracy: 0.9867
Epoch 486/500
41/41 [==============================] - 0s 9ms/step - loss: 0.0521 - accuracy: 0.9867
Epoch 487/500
41/41 [==============================] - 0s 9ms/step - loss: 0.0521 - accuracy: 0.9867
Epoch 488/500
41/41 [==============================] - 0s 10ms/step - loss: 0.0521 - accuracy: 0.9867
Epoch 489/500
41/41 [==============================] - 0s 10ms/step - loss: 0.0521 - accuracy: 0.9867
Epoch 490/500
41/41 [==============================] - 0s 9ms/step - loss: 0.0521 - accuracy: 0.9867
Epoch 491/500
41/41 [==============================] - 0s 10ms/step - loss: 0.0522 - accuracy: 0.98

In [35]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

def predict__class(text):
    fin_text = preprocessing(text)
    xo = cv.fit_transform(fin_text)
    fin_xo = convert_sparse_matrix_to_sparse_tensor(xo)
    ans = ann.predict(tf.sparse.reorder(fin_xo))*10
    return ans

X_test = convert_sparse_matrix_to_sparse_tensor(X_test)

In [36]:
y_pred = ann.predict(tf.sparse.reorder(X_test))
y_pred

18/18 [==============================] - 0s 5ms/step


array([[0.9996228 ],
       [0.99654984],
       [0.01911014],
       [0.99962294],
       [0.01911031],
       [0.01911014],
       [0.4370289 ],
       [0.01911014],
       [0.99961746],
       [0.01916532],
       [0.99962294],
       [0.02449795],
       [0.9995635 ],
       [0.9996229 ],
       [0.04013529],
       [0.01911054],
       [0.01911014],
       [0.99962205],
       [0.01911158],
       [0.6432269 ],
       [0.01911014],
       [0.03788887],
       [0.99962294],
       [0.9996229 ],
       [0.99962187],
       [0.9996214 ],
       [0.01911014],
       [0.01911014],
       [0.01911014],
       [0.02285734],
       [0.01911811],
       [0.02031764],
       [0.01911014],
       [0.01911014],
       [0.01911141],
       [0.9995236 ],
       [0.99962294],
       [0.01913084],
       [0.99962294],
       [0.01911014],
       [0.9996228 ],
       [0.01911014],
       [0.01911014],
       [0.01911034],
       [0.99962264],
       [0.01911014],
       [0.02407908],
       [0.999

In [37]:
y_pred = (y_pred > 0.4)

In [38]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[217  19]
 [ 42 272]]
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       236
           1       0.93      0.87      0.90       314

    accuracy                           0.89       550
   macro avg       0.89      0.89      0.89       550
weighted avg       0.89      0.89      0.89       550



In [ ]:
#ROC and ANOVA, Loss and Accuracy Curve